In [1]:
import requests,time,datetime
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu
import utils.StockSpider as ss

In [14]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [15]:
df = df_hsA

In [2]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# ss.getAnnualReports(dir_path,df,'2017')

ss.getStockDataByType('E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/')

正在获取：沪深A股...请勿关闭浏览器
第1页-第3条...

Traceback (most recent call last):
  File "E:\wangtao\PythonWorkSpace\SpiderSpace\Stock_Companies\utils\StockSpider.py", line 123, in getStockDataByType
    sheet.write(row_index,1,company.text)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py", line 76, in text
    return self._execute(Command.GET_ELEMENT_TEXT)['value']
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py", line 633, in _execute
    return self._parent.execute(command, params)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=78.0.3904.108

WebDriverException: Message: chrome not reachable
  (Session info: chrome=78.0.3904.108)


In [17]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [24]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})

In [25]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于15%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.15) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0) & \
            (row['operate_cash_flow_ps'] > 0):
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [26]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [27]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [28]:
df_target.shape

(86, 19)

In [29]:
df_target = df_target[df_target['PE_D'].apply(lambda x:(x != '未公布') & (x != '0'))]
df_target = df_target[df_target['PE_S'].apply(lambda x:(x != '未公布') & (x != '0'))]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:ss.getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_D']) / x['PE_M']),axis=1)
df_target = df_target[df_target['pe_rate'] < 0.5]

In [30]:
df_target.shape

(5, 21)

In [31]:
df_target.sort_values(by=['pe_rate'],ascending=True)

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare,PE_M,pe_rate
3576,600566,济川药业,21.66,-2.96%,1.76,0.85%,医药生物 — 中药Ⅱ,3.09,10.46,10.140,1.60元,13.06亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降0.24%\t...,23.05%,84.45%,7.00元,8.15亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,白马股,绩优股,",2.19元,27.630,0.366992
262,603579,荣泰健康,28.70,2.83%,1.50,1.47%,家用电器 — 白色家电,2.50,15.68,14.025,1.50元,2.09亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长13.69%\t...,13.44%,29.42%,11.17元,1.40亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,"绩优股,",2.16元,30.605,0.458258
343,600511,国药股份,24.74,2.36%,1.64,2.07%,医药生物 — 医药商业Ⅱ,1.85,12.99,12.767,1.40元,10.71亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长12.71%\...,11.16%,8.27%,13.08元,7.55亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"白马股,",1.42元,26.590,0.480143
3472,000963,华东医药,22.86,-1.80%,1.35,1.14%,医药生物 — 化学制药,3.50,17.97,13.805,1.26元,22.13亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长22.33%\...,20.14%,32.34%,6.66元,17.50亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"二线蓝筹,",0.52元,28.665,0.481598
3251,002174,游族网络,15.90,-1.00%,0.72,1.91%,传媒 — 互联网传媒,2.66,14.00,15.023,0.81元,7.05亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长3.29%\t\...,14.14%,53.90%,5.97元,8.88亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",0.18元,30.890,0.486339


In [57]:
df_target.shape

(98, 20)